In [1]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup as soup

In [2]:
header = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_1)AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36"}

url = "https://www.glassdoor.es/Empleo/espa%C3%B1a-data-analyst-empleos-SRCH_IL.0,6_IN219_KO7,19.htm" #url that I want to access

html = requests.get(url, headers=header)  #Realizar una solicitud HTTP GET a la URL especificada, incluyendo el encabezado

print(html.status_code) #print the code of the response html

bsoj = soup(html.content, 'lxml') #Crear un objeto BeautifulSoup para analizar el contenido HTML de la respuesta

200


In [3]:
#una funcion que hace una request con un parametro de busqueda y return la request
"""This function makes a request with a search parameter
   and returns the resulting BeautifulSoup object.
"""

def request_data(url):
    header = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_1)AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36"}
    # Definir el encabezado (header) para la solicitud HTTP
    html = requests.get(url, headers=header) #Realizar una solicitud HTTP GET a la URL especificada, incluyendo el encabezado
    if html.status_code == 200:
        bsoj = soup(html.content, 'lxml') #Crear un objeto BeautifulSoup para analizar el contenido HTML de la respuesta
        return bsoj
    else:
        print (f"None: {url}")

In [4]:
# primero una funcion que tenga parametros : el resultado de una request y return un df
"""This function takes a BeautifulSoup object as a parameter and creates a DataFrame.
"""

def create_data_frame(bsoj):
    #Create an empty DataFrame with specified column names
    df = pd.DataFrame(columns=["id", "company","puesto","locality"])
    
    # Initialize a dictionary to store job data
    job_data = {}
    
    #Find the container element that holds job listings
    un_container = bsoj.find("div", class_ ="JobsList_wrapper__EyUF6") 

    # Find all job list items within the container
    job_list_items = un_container.find_all("li", class_="JobsList_jobListItem__wjTHv") 
    
    #Iterate through each job list item in the container
    id = 0000
    for i in job_list_items[:-1]:  # Exclude the last item
        # Extract job details from the current job list item
        job_data["company"] = i.find("div", class_="EmployerProfile_profileContainer__VjVBX EmployerProfile_compact__nP9vu").text.strip()
        job_data["puesto"] = i.find("a", class_="JobCard_jobTitle___7I6y").text.strip()
        job_data["locality"] = i.find("div", class_="JobCard_location__rCz3x").text.strip()
        # Increment id for each job
        id += 1
        job_data["id"] = id
        
        # Convert the job data dictionary to a DataFrame
        new_df = pd.DataFrame([job_data])

        # Concatenate the new DataFrame to the existing DataFrame
        df = pd.concat([df, new_df], ignore_index=True)
    return df

In [10]:
one_request = request_data("https://www.glassdoor.es/Empleo/espa%C3%B1a-data-analyst-empleos-SRCH_IL.0,6_IN219_KO7,19.htm") 

In [11]:
df_prueba = create_data_frame(one_request)
df_prueba.sample()

,id,company,puesto,locality
22,23,"Rank International3,6",Data Analyst,Barcelona


In [5]:
lista_de_busquedas = ["data-analyst", "data-science", "data-engineer"]

df = pd.DataFrame(columns=["id", "company","puesto","locality"])

for busqueda in lista_de_busquedas:
    
    url = f"https://www.glassdoor.es/Empleo/espa%C3%B1a-{busqueda}-empleos-SRCH_IL.0,6_IN219_KO7,13.htm"

    one_request = request_data(url)
    
    df = pd.concat([df, create_data_frame(one_request)], ignore_index=True)

df

,id,company,puesto,locality
0,1,"DataForce by TransPerfect3,1",Dutch Language Data Annotator - Remote Spain,España
1,2,Nord Anglia Education,HR Data Analyst,Trabajo en remoto
2,3,"DataForce by TransPerfect3,1",Language Data Annotator/Content Generator Germ...,Barcelona
3,4,Polinex Inc.,Full-Stack Developer & Digital Marketing Speci...,Trabajo en remoto
4,5,"EMR CPR LLC3,4",Data Center Engineer,España
...,...,...,...,...
85,26,"Bosch Group4,1","E-commerce Excellence Hub Specialist (EEHS), F...",Barcelona
86,27,"EcoVadis4,3",Junior Workday Reporting Analyst (HCM),Barcelona
87,28,"Candee5,0",E-commerce Specialist (Shopify) at Tropicfeel,Barcelona
88,29,"Enel3,9",BECAS ENDESA – FERIA TERUEL EMPLEA,España


In [6]:
df.to_excel('glassdoor_fin.xlsx', index=False)